## Install

In [2]:
%%capture
!pip install pip3-autoremove
!pip-autoremove torch torchvision torchaudio -y
!pip install torch torchvision torchaudio xformers --index-url https://download.pytorch.org/whl/cu121
!pip install unsloth

In [3]:
!pip install -q datasets

## Config

In [4]:
TEST_DATA_PATH = "/kaggle/input/vcs-b1/B1.csv"
MODEL_PATH = "/kaggle/input/vcs-lora-model"

## Prepare dataset

In [30]:
from datasets import load_dataset
test_dataset = load_dataset('csv', data_files=TEST_DATA_PATH)['train']

In [31]:
test_dataset

Dataset({
    features: ['index', 'question', 'ground_truth'],
    num_rows: 150
})

## Load model

In [7]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [8]:
from unsloth import FastLanguageModel
model_infer, tokenizer_infer = FastLanguageModel.from_pretrained(
    model_name = MODEL_PATH, # YOUR MODEL YOU USED FOR TRAINING
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = False,
    #load_in_4bit = load_in_4bit,
)
FastLanguageModel.for_inference(model_infer) # Enable native 2x faster inference

==((====))==  Unsloth 2024.12.11: Fast Llama patching. Transformers: 4.47.1.
   \\   /|    GPU: Tesla P100-PCIE-16GB. Max memory: 15.888 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 6.0. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

Unsloth 2024.12.11 patched 16 layers with 16 QKV layers, 16 O layers and 16 MLP layers.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 2048, padding_idx=128004)
        (layers): ModuleList(
          (0-15): 16 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=128, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=128, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Line

## Inference

In [9]:
def infer_template(dataset, index):
    """
    Tạo một chuỗi văn bản với định dạng chuẩn, bao gồm câu hỏi và các lựa chọn từ một bản ghi trong dataset.
    
    Args:
        dataset (Dataset): Dữ liệu đầu vào chứa các câu hỏi và lựa chọn.
        index (int): Chỉ số của phần tử trong dataset cần được xử lý.
    
    Returns:
        str: Một chuỗi văn bản đã được định dạng với câu hỏi và các lựa chọn.
    
    Example:
        infer_template(dataset, 0)
        # Trả về một chuỗi văn bản với câu hỏi và các lựa chọn của phần tử tại chỉ số 0 trong dataset.
    """
    
    # Lấy câu hỏi từ phần tử dataset
    question_end_index = dataset[index]['question'].find('A.')
    question = dataset[index]['question'][:question_end_index]
    
    # Lấy các lựa chọn (choices) từ phần tử dataset
    choices = dataset[index]['question'][question_end_index:].replace('  ', ' ')
    
    # Định dạng văn bản đầu ra
    text = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are a cybersecurity assistant answering multiple-choice questions on ethical hacking and cybersecurity practices, specifically aligned with CEH v10 standards. Return only the correct answer without providing explanations.
<|eot_id|><|start_header_id|>user<|end_header_id|>

Question: {question}
Choices: 
{choices}
<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""

    return text


In [10]:
def inference(input):
    """
    Perform inference using the model and tokenizer.

    Parameters:
        input (str): The user input to be processed.

    Returns:
        str: The decoded output from the model.
    """
    # Tokenize input and move tensors to GPU
    inputs = tokenizer_infer(input, return_tensors="pt", add_special_tokens=True).to("cuda")

    # Generate output from the model
    outputs = model_infer.generate(
        input_ids=inputs["input_ids"],
        max_new_tokens=64,
        use_cache=True,
        temperature=0.1,
        min_p=0.1
    )

    # Decode and return the result
    return tokenizer_infer.decode(outputs[0], skip_special_tokens=True)

In [11]:
from tqdm import tqdm

def process_dataset(test_dataset, infer_template, inference):
    """
    Process a dataset and collect inference results.

    Parameters:
        test_dataset (list): The dataset to process.
        infer_template (function): A function to generate input from the dataset.
        inference (function): A function to perform inference on the input.

    Returns:
        list: A list of inference results.
    """
    from tqdm import tqdm

    results = []

    # Lặp qua toàn bộ dataset với tqdm
    for index in tqdm(range(len(test_dataset)), desc="Processing Dataset"):
        input_sample = infer_template(test_dataset, index)
        result = inference(input_sample)
        results.append(result)

    return results

# Ví dụ sử dụng:
results = process_dataset(test_dataset, infer_template, inference)

Processing Dataset: 100%|██████████| 150/150 [04:03<00:00,  1.62s/it]


In [27]:
print(results[0])

system
You are a cybersecurity assistant answering multiple-choice questions on ethical hacking and cybersecurity practices, specifically aligned with CEH v10 standards. Return only the correct answer without providing explanations.
user

Question: You have successfully logged on a Linux system. You want to now cover your trade Your login attempt may
be logged on several files located in /var/log. Which file does NOT belongs to the list:

Choices: 
A. user.log
B. auth.fesg
C. wtmp
D. btmp

assistant
The correct answer: A. user.log
Reasoning: The user.log file belongs to the user and is typically used for logging user-related events. The other files are system logs that belong to the system and are used for system-wide logging purposes.


In [28]:
def extract_answers(results, full_answer=False):
    """
    Extract answers from a list of results.

    Parameters:
        results (list of str): List of result strings to extract answers from.
        full_answer (bool): If True, extract the full answer. If False, extract only the representative letter.

    Returns:
        list of str: Extracted answers.
    """
    answers = []
    for result in results:
        # Tìm vị trí bắt đầu của "The correct answer: "
        start_index = result.find("The correct answer: ") + len("The correct answer: ")
        
        if full_answer:
            # Tìm vị trí kết thúc của đáp án (đến dấu xuống dòng hoặc kết thúc chuỗi)
            end_index = result.find("\n", start_index)
            if end_index == -1:  # Nếu không có dấu xuống dòng, lấy đến hết chuỗi
                end_index = len(result)
            # Lấy nội dung đáp án đầy đủ
            answer = result[start_index:end_index].strip()
        else:
            # Chỉ lấy chữ cái đại diện (ký tự đầu tiên sau "The correct answer: ")
            answer = result[start_index].strip()

        answers.append(answer)
    return answers

def extract_response(results):
    """
    Extract responses from a list of results starting from a specific marker.

    Parameters:
        results (list of str): List of result strings to extract responses from.

    Returns:
        list of str: Extracted responses.
    """
    responses = []
    marker = "\nassistant\n"

    for result in results:
        # Tìm vị trí bắt đầu của marker
        start_index = result.find(marker) + len(marker)
        
        if start_index != -1:
            # Lấy nội dung từ vị trí marker đến hết
            response = result[start_index:].strip()
            responses.append(response)
        else:
            # Nếu không tìm thấy marker, thêm thông báo rỗng hoặc tùy chọn khác
            responses.append("")

    return responses

def check_answers(example):
    return {'is_correct': 1 if example['ground_truth'] == example['predicted_answers'] else 0}

In [32]:
predicted_response = extract_response(results) # Danh sách các câu trả lời đầy đủ dự đoán
predicted_answers = extract_answers(results)  # Danh sách các đáp án chữ cái đại diện dự đoán

In [33]:
print("First response: ", predicted_response[0], "\n", "-"*150, "\n")
print("First ans: ", predicted_answers[0])

First response:  The correct answer: A. user.log
Reasoning: The user.log file belongs to the user and is typically used for logging user-related events. The other files are system logs that belong to the system and are used for system-wide logging purposes. 
 ------------------------------------------------------------------------------------------------------------------------------------------------------ 

First ans:  A


In [34]:
# Thêm cột "predicted_response" vào dataset
test_dataset = test_dataset.add_column("predicted_response", predicted_response)

# Thêm cột "predicted_answers" vào dataset
test_dataset = test_dataset.add_column("predicted_answers", predicted_answers)

# So sánh "predicted_answers" và "ground_truth" để thêm cột "is_correct" vào dataset
test_dataset = test_dataset.map(check_answers)

Map:   0%|          | 0/150 [00:00<?, ? examples/s]

In [35]:
test_dataset

Dataset({
    features: ['index', 'question', 'ground_truth', 'predicted_response', 'predicted_answers', 'is_correct'],
    num_rows: 150
})

In [36]:
def calculate_accuracy(dataset, column_name="is_correct"):
    """
    Calculate the accuracy based on a specified column in the dataset.

    Parameters:
        dataset (dict or DataFrame): The dataset containing the results.
        column_name (str): The name of the column containing boolean correctness values.

    Returns:
        float: The accuracy percentage.
    """
    # Lấy danh sách giá trị của cột
    check_values = dataset[column_name]

    # Tính số câu đúng
    num_correct = sum(check_values)
    
    # Tính accuracy
    accuracy = round((num_correct / len(check_values)) * 100, 2)

    return num_correct, accuracy

# Tính accuracy
num_correct, accuracy = calculate_accuracy(test_dataset)
print("Số câu đúng: ", num_correct)
print("Accuracy: ", accuracy)

Số câu đúng:  60
Accuracy:  40.0


In [37]:
import pandas as pd

def save_dataset(dataset, filename="output.xlsx", file_format="excel"):
    """
    Save a dataset to an Excel or CSV file.

    Parameters:
        dataset (list or dict): The dataset to be converted into a DataFrame.
        filename (str): The name of the file to save.
        file_format (str): The format of the file ("excel" or "csv").

    Returns:
        None
    """
    # Chuyển đổi dataset thành DataFrame
    df = pd.DataFrame(dataset)

    if file_format.lower() == "excel":
        # Lưu DataFrame thành file Excel
        df.to_excel(filename, index=False)
        print(f"Dataset đã được lưu thành file Excel: {filename}")
    elif file_format.lower() == "csv":
        # Lưu DataFrame thành file CSV
        df.to_csv(filename, index=False)
        print(f"Dataset đã được lưu thành file CSV: {filename}")
    else:
        raise ValueError("file_format phải là 'excel' hoặc 'csv'")

# Lưu ra file excel hoặc csv
save_dataset(test_dataset, filename="output.xlsx", file_format="excel")
# save_dataset(test_dataset, filename="output.xlsx", file_format="csv")

Dataset đã được lưu thành file Excel: output.xlsx
